# Coleta das *Fake News* reportadas pelo portal Boatos.org

- Fontes: https://boatos.org - Boatos.org


- Objetivo: Coletar as *Fake News* reportadas nas últimas 10 páginas da categoria 'Brasil' do portal Boatos.org.

## Importando as bibliotecas e definindo as constantes e variáveis iniciais

In [1]:
import requests
from bs4 import BeautifulSoup

PORTAL = "https://www.boatos.org/category/brasil/"

data = []

# List of the URL from the last 10 pages of the portal.
portal_pages = []
for i in range(10):
    if i == 0:
        portal_pages.append(PORTAL)
    else:
        portal_pages.append(PORTAL + f'page/{i+1}')
portal_pages

['https://www.boatos.org/category/brasil/',
 'https://www.boatos.org/category/brasil/page/2',
 'https://www.boatos.org/category/brasil/page/3',
 'https://www.boatos.org/category/brasil/page/4',
 'https://www.boatos.org/category/brasil/page/5',
 'https://www.boatos.org/category/brasil/page/6',
 'https://www.boatos.org/category/brasil/page/7',
 'https://www.boatos.org/category/brasil/page/8',
 'https://www.boatos.org/category/brasil/page/9',
 'https://www.boatos.org/category/brasil/page/10']

## Coletando os links individuais de cada reportagem de *Fake News*

In [5]:
fake_news_pages = []
for link in portal_pages:
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    link_tags = soup.select('div.article-container h2.entry-title a')
    for a in link_tags:
        fake_news_pages.append(a['href'])

# Check first 10 links and show total number of links
print(fake_news_pages[:10])
print(f'Total: {len(fake_news_pages)} páginas')

['https://www.boatos.org/brasil/mcdonalds-vai-dar-dois-big-macs-promocao-vassoura-thru.html', 'https://www.boatos.org/brasil/mulher-pisou-cachorro-morta-bandidos.html', 'https://www.boatos.org/brasil/brasil-tem-menor-salario-minimo-america-do-sul-argentina-tem-maior.html', 'https://www.boatos.org/brasil/maguila-dois-anos-asilo-sem-receber-visita-esposa-filhos.html', 'https://www.boatos.org/brasil/macron-franca-anexar-amapa-guiana-francesa.html', 'https://www.boatos.org/brasil/policiais-brasil-greve-repudio-stf-soltura-andre-do-rap.html', 'https://www.boatos.org/brasil/foto-primeiro-bebe-johnson-hoje.html', 'https://www.boatos.org/brasil/lider-ong-mst-dono-madeira-apreendida-exercito-para.html', 'https://www.boatos.org/brasil/celso-mello-antecipou-aposentadoria-invalidez-stf-nao-pagar-imposto.html', 'https://www.boatos.org/brasil/ladra-ativista-ong-presa-aeroporto-roubar-biodiversidade-amazonia.html']
Total: 500 páginas


## Coletando as *Fake News*

Coleta os textos de *fake news* em uma lista de dicionários onde:
- 'news': é o texto da notícia.
- 'fake': 0 caso a notícia seja verdadeira ou 1, caso falsa.

> Neste caso, todas as notícias são falsas e, portanto, *'fake'* será sempre igual a 1. O motivo da existencia da variável *'fake'* é para facilitar a união destes dados com um de notícias verdadeiras no futuro. Um *dataset* equilibrado de notícias falsas e verdadeiras, devidamente classificadas, pode ser utilizado, com auxílio de técnicas de NLP, na criação de uma aplicação que identifica, através de um dado texto, se trata-se de uma notícia falsa ou não.

> Algumas notícias não possuem um texto associado (por exemplo, no caso de notícias em forma de vídeo ou audio), outras possuem mais de uma versão do texto. Por este motivo o número de *fake news* coletadas pode diferir do número de páginas coletadas na etapa anterior.

In [8]:
for report_page in fake_news_pages:
    page = requests.get(report_page)
    soup = BeautifulSoup(page.content, 'html.parser')

    fake_news = soup.select('span em')
    for fake_text in fake_news:
        if fake_text.get_text():
            data.append(
                {
                    "news": fake_text.get_text(),
                    "fake": 1
                }
            )
print(data[:10])
print(f'Total: {len(data)} fake news')

[{'news': 'A companhia esclarece que qualquer comunicação relacionada ao Halloween estimulando os consumidores a passarem pelo drive-thru dos restaurantes da rede com vassouras é falsa e não há nenhuma ação promocional da marca nesse formato.', 'fake': 1}, {'news': 'Tanta maldade com um ser tão amável e carente .ela pisou na cabeça de um cachorro por pura maldade .cadeia nessa verme. ', 'fake': 1}, {'news': 'Salários mínimos oficiais na América do Sul. 1. Argentina – 2.826 reais 2. Venezuela – 2.593 reais. 3. Equador – 2.497 reais 4. Uruguai – 2.050 reais 5. Colômbia – 1.977 reais 6. Bolívia – 1.905 reais 7. Paraguai – 1.880 reais 8. Peru – 1.877 reais 9. Chile – 1.733 reais 10. Suriname 1.682 reais 11. Bolívia – 1.465 reais 12. Guiana – 1.171 reais 13. Brasil – 1.045 reais (Fonte: Banco Mundial, outubro 2020. ', 'fake': 1}, {'news': '“SERÁ QUE É VERDADE??? E ai? Mais uma vez as FA irão limpar a m* que os outros fazem?\xa0 REPASSANDO COMO RECEBI. PODE TER FUNDO DE VERDADE. Soltura de A

## Salvando os dados em um arquivo CSV

In [9]:
import csv

keys = data[0].keys()
with open('fake_news.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data)